In [38]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import h5py

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers.legacy import SGD
import random
from keras.models import load_model

# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# importing the GL Bot corpus file for pre-processing

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("data.json").read()
intents = json.loads(data_file)

In [39]:
# preprocessing the json data
# tokenization
#nltk.download('punkt')
#nltk.download('wordnet')
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [40]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

# creating a pickle file to store the Python objects which we will use while predicting
pickle.dump(words,open('texts.pkl','wb')) 
pickle.dump(classes,open('labels.pkl','wb'))

232 documents
93 classes ['acceleration_problem', 'accessory_issue', 'battery_lifespan', 'battery_problem', 'body_damage', 'body_paint_issue', 'brake_check', 'brake_fluid', 'brake_issue', 'cable_check', 'carburator_issue', 'chain_drive_maintenance', 'chain_issue', 'chain_lubrication', 'chain_tension', 'check_bearings', 'clutch_adjustment', 'clutch_cable_issue', 'clutch_issue', 'common_maintenance_errors', 'cooling_fluid', 'cooling_system', 'emergency_setup', 'engine_noises', 'engine_problem', 'exhaust_check', 'exhaust_issue', 'first_aid_knowledge', 'flashlight_check', 'flooded_engine', 'fork_oil_change', 'fuel_conserving_tips', 'fuel_filter_issue', 'fuel_leak', 'fuel_quality', 'fuel_system_cleaning', 'fuel_tank_check', 'fuel_time_check', 'fuel_warning', 'fuel_warning_light', 'gasoline_quality', 'generic_help', 'greeting', 'handgrip_issue', 'headlight_issue', 'horn_issue', 'idle_issue', 'idling_issue', 'insurance_claim', 'lighting_issue', 'long_term_storage', 'lost_key_service', 'mainte

In [41]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
   
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

Training data created


In [42]:
# Define NN model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model using legacy SGD
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fit and save the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('models.h5')

print("\n")
print("*" * 50)
print("\nModel Created Successfully!")

Epoch 1/200
47/47 [==============================] - 0s 1ms/step - loss: 4.5605 - accuracy: 0.0043
Epoch 2/200
47/47 [==============================] - 0s 885us/step - loss: 4.5356 - accuracy: 0.0129
Epoch 3/200
47/47 [==============================] - 0s 1ms/step - loss: 4.5154 - accuracy: 0.0172
Epoch 4/200
47/47 [==============================] - 0s 973us/step - loss: 4.4769 - accuracy: 0.0345
Epoch 5/200
47/47 [==============================] - 0s 1ms/step - loss: 4.4376 - accuracy: 0.0560
Epoch 6/200
47/47 [==============================] - 0s 1ms/step - loss: 4.4295 - accuracy: 0.0603
Epoch 7/200
47/47 [==============================] - 0s 956us/step - loss: 4.3798 - accuracy: 0.0733
Epoch 8/200
47/47 [==============================] - 0s 945us/step - loss: 4.3400 - accuracy: 0.0431
Epoch 9/200
47/47 [==============================] - 0s 1ms/step - loss: 4.2420 - accuracy: 0.0733
Epoch 10/200
47/47 [==============================] - 0s 973us/step - loss: 4.1861 - accuracy: 0.0690

In [43]:
loss, accuracy = model.evaluate(np.array(train_x), np.array(train_y), verbose=1)
print(f"Loss: {loss}, Accuracy: {accuracy}")

8/8 [==============================] - 0s 944us/step - loss: 0.0118 - accuracy: 0.9957
Loss: 0.0117875337600708, Accuracy: 0.9956896305084229
